In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

In [2]:
(ds_train, ds_test), ds_info = tfds. load (
    "cifar10",
    split=["train", "test"],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,

)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/3.0.2.incompleteH4WL3N/cifar10-train.tfrecord*...:   0%|          …

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/3.0.2.incompleteH4WL3N/cifar10-test.tfrecord*...:   0%|          |…

Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.


In [3]:
def normalize_image(image, label):
  return tf.cast(image, tf.float32)/255.0, label

In [4]:
def augment (image, label):
  new_height = new_width = 32

  image = tf.image.resize(image,(new_height, new_width))

  if tf.random.uniform((), minval=0, maxval=1) < 0.1:
    image = tf.image.rgb_to_grayscale (image)
    image=tf.tile(image, [1,1,3])
  image = tf.image.random_brightness(image, max_delta=0.1)
  image = tf. image. random_contrast (image, lower=0.1, upper=0.2)
  image = tf.image.random_flip_left_right (image)
  return image, label

In [5]:
AUTOTUNE=tf.data.experimental.AUTOTUNE #tf allow us to give this to set automatically
BATCH_SIZE=32

ds_train=ds_train.map(normalize_image, num_parallel_calls=AUTOTUNE) # in normalize function these is no sequence so
#process can be done paralelly
ds_train= ds_train.cache() #it keep next data in memory so it will be fast
ds_train=ds_train.shuffle(ds_info.splits['train'].num_examples) #we can give 1000 so it will see 1000 datat points but
#it depends on records in a file so take from that


#ds_train=ds_train.map(augment, num_parallel_calls=AUTOTUNE)



ds_train=ds_train.batch(BATCH_SIZE)
ds_train=ds_train.prefetch(AUTOTUNE) #it keeps 64 examples ready when running on gpu the current one

In [6]:
ds_test=ds_test.map(normalize_image, num_parallel_calls=AUTOTUNE)
ds_test=ds_test.batch(128)
ds_test=ds_test.prefetch(AUTOTUNE)

In [7]:
class_names = [
"Airplane",
"Autmobile",
"Bird",
"Cat",
"Deep",
"Dog",
"Frog",
"Horse"
"Ship",
"Truck"
]

In [8]:
def get_model() :
  model = keras. Sequential (
      [
          layers. Input ((32, 32, 3)),
          layers. Conv2D(8, 3, padding ="same", activation="relu"),
          layers. Conv2D(16, 3, padding="same", activation="relu"),
          layers. MaxPooling2D((2, 2)),
          layers .Flatten (),
          layers. Dense (64, activation="relu"),
          layers.Dropout (0.1),
          layers. Dense (10),
]
  )
  return model

In [9]:
#Using tensorboard using callbacks for logging data
model = get_model ()

model. compile(
    optimizer=keras.optimizers.Adam(lr=0.001),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

tensorboard_callback = keras.callbacks.TensorBoard(
    log_dir= 'tb_callbacks_dir', #where we want to keep track all logs
    histogram_freq=1, #for distribution plots of gradients
)

In [10]:
#for running tensorboard from colab
#we dont need to do all this when doing on system

#loading tensorboard
%load_ext tensorboard

#The tensorboard extension is already loaded. To reload it, use:
#%reload_ext tensorboard

#remove log folder
!rm -rf ./tb_callbacks_dir/

import datetime
log_ = "tb_callbacks_dir/modelFit/" + datetime.datetime.now().strftime("%d-%m-%Y-%H%M%S")


tensorboard_callback = keras.callbacks. TensorBoard(
    log_dir= log_,
    histogram_freq=1,
)

In [11]:
model.fit(
    ds_train,
    epochs=2,
    validation_data=ds_test,
    callbacks=[tensorboard_callback],
    verbose=2,
)

Epoch 1/2
1563/1563 - 27s - loss: 1.5476 - accuracy: 0.4415 - val_loss: 1.3420 - val_accuracy: 0.5147 - 27s/epoch - 17ms/step
Epoch 2/2
1563/1563 - 6s - loss: 1.2429 - accuracy: 0.5574 - val_loss: 1.1577 - val_accuracy: 0.5883 - 6s/epoch - 4ms/step


In [12]:
#when doing on colab then we use tensorboard extension

# %tensorboard --logdir tb_callbacks_dir/modelFit/

In [13]:
#this we do in command line when working on system

#tensorboard --logdir tb_callbacks_dir

In [14]:
#scalar show train val accuracy loss
#graph show the structure of model
#histogram show distribution of parameters

In [15]:
#custom traning and as it gives more flexibity
#here we are creating logs in custom training and using tensorboard

model = get_model ()
num_epochs = 5
loss_fn = keras. losses. SparseCategoricalCrossentropy (from_logits=True)
optimizer = keras.optimizers.Adam (lr=0.001)
acc_metric = keras .metrics. SparseCategoricalAccuracy ()

#these are writers to write things
train_writer = tf.summary.create_file_writer ("logs/train/")
test_writer = tf.summary. create_file_writer ("logs/test/")
train_step = test_step = 0 #for train and test both will be tracked at every step

In [16]:
for epoch in range (num_epochs) :
  # Iterate through training set
  for batch_idx, (x, y) in enumerate (ds_train) :

    with tf. GradientTape () as tape:
      y_pred = model (x, training=True)
      loss = loss_fn(y, y_pred)

    gradients = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(gradients, model.trainable_weights))
    acc_metric.update_state(y, y_pred)

  with train_writer.as_default():
    tf.summary.scalar('Loss', loss, step=epoch) #here we are giving loss of last batch instead we have to make a list append all and give mean of that
    tf.summary.scalar('Accuracy', acc_metric.result(), step=epoch) #it got reset for every epoch for train and test

  # Reset accuracy in between epochs (and for testing and test)
  acc_metric.reset_states ()


  # Iterate through test set
  for batch_idx, (x,y) in enumerate (ds_test):
    y_pred = model(x, training=False)
    loss = loss_fn(y, y_pred)
    acc_metric.update_state (y, y_pred)

  with test_writer.as_default():
    tf.summary.scalar('Loss', loss, step=epoch) #here we are giving loss of last batch instead we have to make a list append all and give mean of that
    tf.summary.scalar('Accuracy', acc_metric.result(), step=epoch) #it got reset for every epoch for train and test


  acc_metric.reset_states ()

In [17]:
# %reload_ext tensorboard
# %tensorboard --logdir logs/

In [18]:
#we can also log things for every batch so graph will be more smoother

In [ ]:
#we can do hyperparameter tuning with custom looping
#here doing grid search

for lr in [1e-1, 1e-2, 1e-3, 1e-4]:
  train_writer = tf.summary.create_file_writer ("logs/train/"+str(lr))
  test_writer = tf.summary. create_file_writer ("logs/test/"+str(lr))
  train_step = test_step = 0
  model = get_model ()
  optimizer = keras.optimizers.Adam (lr=lr)

  for epoch in range (num_epochs) :
    # Iterate through training set
    for batch_idx, (x, y) in enumerate (ds_train) :

      with tf. GradientTape () as tape:
        y_pred = model (x, training=True)
        loss = loss_fn(y, y_pred)

      gradients = tape.gradient(loss, model.trainable_weights)
      optimizer.apply_gradients (zip (gradients, model.trainable_weights))
      acc_metric.update_state (y, y_pred)

      with train_writer.as_default():
        tf.summary.scalar('Loss', loss, step=train_step)  #for every batch we are doing
        tf.summary.scalar('Accuracy', acc_metric.result(), step=train_step)
        train_step+=1

    # Reset accuracy in between epochs (and for testing and test)
    acc_metric.reset_states ()


    # Iterate through test set
    for batch_idx, (x,y) in enumerate (ds_test):
      y_pred = model(x, training=False)
      loss = loss_fn(y, y_pred)
      acc_metric.update_state (y, y_pred)

      with test_writer.as_default():
        tf.summary.scalar('Loss', loss, step=test_step)
        tf.summary.scalar('Accuracy', acc_metric.result(), step=test_step)
        test_step+=1


    acc_metric.reset_states ()

In [ ]:
# %reload_ext tensorboard
# %tensorboard --logdir logs/

### Visualizing images

In [ ]:
#visualizing images

In [ ]:
def augment (image, label):
  new_height = new_width = 32

  image = tf.image.resize(image,(new_height, new_width))

  if tf.random.uniform((), minval=0, maxval=1) < 0.1:
    image = tf.image.rgb_to_grayscale (image)
    image=tf.tile(image, [1,1,3])
  image = tf.image.random_brightness(image, max_delta=0.1)
  image = tf. image. random_contrast (image, lower=0.1, upper=0.2)
  image = tf.image.random_flip_left_right (image)

  #matplotlib wants value between 0 and 1
  #though we nomalize image but doing augmentation can happen that values are not in [0,1]
  image=tf.clip_by_value(image, clip_value_min=0, clip_value_max=1)

  return image, label

In [ ]:
AUTOTUNE=tf.data.experimental.AUTOTUNE
BATCH_SIZE=32

ds_train=ds_train.map(normalize_image, num_parallel_calls=AUTOTUNE)
ds_train= ds_train.cache()
ds_train=ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train=ds_train.map(augment, num_parallel_calls=AUTOTUNE)
ds_train=ds_train.batch(BATCH_SIZE)
ds_train=ds_train.prefetch(AUTOTUNE)

ds_test=ds_test.map(normalize_image, num_parallel_calls=AUTOTUNE)
ds_test=ds_test.batch(128)
ds_test=ds_test.prefetch(AUTOTUNE)

class_names = [
"Airplane",
"Autmobile",
"Bird",
"Cat",
"Deep",
"Dog",
"Frog",
"Horse",
"Ship",
"Truck"
]

In [ ]:
!rm -rf ./image_logs/

In [ ]:
model = get_model ()
num_epochs = 1
loss_fn = keras. losses. SparseCategoricalCrossentropy (from_logits=True)
optimizer = keras.optimizers.Adam (lr=0.001)
acc_metric = keras .metrics. SparseCategoricalAccuracy ()

writer = tf.summary.create_file_writer ("/content/image_logs/")
step = 0

In [ ]:
#plot_to_image and image_grid should be defined in utils.py
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import numpy as np
import io
import sklearn.metrics
from tensorboard.plugins import projector
import cv2
import os
import shutil

def plot_to_image(figure):
  """Converts the matplotlib plot specified by 'figure' to a PNG image and
  returns it. The supplied figure is closed and inaccessible after this call."""
  # Save the plot to a PNG in memory.
  buf = io.BytesIO()
  plt.savefig(buf, format='png')
  # Closing the figure prevents it from being displayed directly inside
  # the notebook.
  plt.close(figure)
  buf.seek(0)
  # Convert PNG buffer to TF image
  image = tf.image.decode_png(buf.getvalue(), channels=4)
  # Add the batch dimension
  image = tf.expand_dims(image, 0)
  return image


def image_grid(data, labels, class_names):

  #data should be in BATCH_SIZE*H*W*C form
  assert data.ndim==4

  """Return a 5x5 grid as a matplotlib figure."""

  # Create a figure to contain the plot.
  figure = plt.figure(figsize=(10,10))

  num_images=data.shape[0]
  size=int(np.ceil(np.sqrt(num_images)))

  for i in range(num_images):
    # Start next subplot.
    plt.subplot(size,size, i+1, title=class_names[labels[i]])
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)

    if data.shape[3]==1:
      plt.imshow(data[i], cmap=plt.cm.binary)
    else:
      plt.imshow(data[i], cmap=plt.cm.binary)

  return figure

In [ ]:
sampled_train = ds_train.take(5)

In [ ]:
for epoch in range (num_epochs) :
  # Iterate through training set
  for batch_idx, (x, y) in enumerate (sampled_train) :

    figure = image_grid(x, y, class_names)
    with writer.as_default() :
      tf.summary.image ("Visualize Images", plot_to_image (figure), step=step,)
    step += 1

In [ ]:
# %reload_ext tensorboard
# %tensorboard --logdir image_logs/

### Confusion matrix

In [ ]:
#confusion matrix

In [ ]:
#create get_confusion_matrix and plot_confusion_matrix in utils.py
import numpy as np
from sklearn import metrics
def get_confusion_matrix (y_labels, logits, class_names):
  preds=np.argmax(logits, axis=1)
  cm= metrics.confusion_matrix(
      y_labels, preds, labels=np.arange(len(class_names)) #we write labels as in batch can happen that some classes not theere
      #so it will destroy whole matrix
  )
  return cm
def plot_confusion_matrix(cm, class_names) :

  size = len (class_names)
  figure = plt.figure(figsize=(size, size))
  plt.imshow (cm, interpolation="nearest", cmap=plt.cm.Blues)
  plt.title ("Confusion Matrix")
  indices = np.arange (len (class_names))
  plt.xticks (indices, class_names, rotation=45)
  plt.yticks (indices, class_names)

  # Normalize Confusion Matrix
  cm = np. around (
      cm.astype ("float") / cm.sum (axis=1) [:, np. newaxis], decimals=3,
      )
  threshold = cm.max() / 2.0
  for i in range (size):
    for j in range (size) :
      color = "white" if cm[i,j] > threshold else "black"
      plt.text(
          i, j, cm[i, j], horizontalalignment="center", color=color,
      )
  plt.tight_layout ()
  plt.xlabel("True Label")
  plt.ylabel("Predicted label")
  cm_image = plot_to_image (figure)
  return cm_image

In [ ]:
!rm -rf ./confusionMat_logs/

In [ ]:
model = get_model ()
num_epochs = 1
loss_fn = keras. losses. SparseCategoricalCrossentropy (from_logits=True)
optimizer = keras.optimizers.Adam (lr=0.001)
acc_metric = keras .metrics. SparseCategoricalAccuracy ()

train_writer = tf.summary.create_file_writer ("confusionMat_logs/train/")
test_writer = tf.summary. create_file_writer ("confusionMat_logs/test/")
train_step= test_step = 0


#at every batch we will be updating confusion matrix
for epoch in range (num_epochs) :
  confusion=np.zeros((len(class_names), len(class_names)))

  for batch_idx, (x, y) in enumerate (sampled_train) :

    with tf. GradientTape () as tape:
      y_pred = model (x, training=True)
      loss = loss_fn(y, y_pred)

    gradients = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients (zip (gradients, model.trainable_weights))
    acc_metric.update_state (y, y_pred)

    confusion +=get_confusion_matrix(y, y_pred, class_names)

    with train_writer.as_default() :
      tf.summary. image ("Confusion Matrix",
                         plot_confusion_matrix(confusion/batch_idx, class_names), #because we want
                         #average at every epoch so divide by batch idx
                         step=epoch,)

  acc_metric.reset_states ()

In [ ]:
# %reload_ext tensorboard
# %tensorboard --logdir confusionMat_logs/

### Visulaizing graphs

In [ ]:
#graph vis

In [ ]:
!rm -rf ./logs/

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers


writer = tf. summary. create_file_writer ("logs/graphs_vis")

In [ ]:
model=get_model()

In [ ]:
model.summary()

In [ ]:
@tf.function #for telling that make this part of static graph
def my_func(x, y) :
  return tf.nn.relu(tf .matmul (x, y))

x = tf.random. uniform ( (3, 3))
y = tf.random. uniform ( (3, 3))

tf.summary.trace_on(graph=True, profiler=True)
out=my_func(x,y)

In [ ]:
with writer.as_default():
  tf.summary.trace_export (
      name="function_trace",
      step=0,
      profiler_outdir="logs/graphs_vis/"
  )

In [ ]:
# %reload_ext tensorboard
# %tensorboard --logdir logs/

### Hyperparameter

In [ ]:
#Hyperparameter

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np


from tensorboard.plugins.hparams import api as hp

In [ ]:
(ds_train, ds_test), ds_info = tfds. load (
    "cifar10",
    split=["train", "test"],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,

)
def normalize_image(image, label):
  return tf.cast(image, tf.float32)/255.0, label
def augment (image, label):
  new_height = new_width = 32

  image = tf.image.resize(image,(new_height, new_width))

  if tf.random.uniform((), minval=0, maxval=1) < 0.1:
    image = tf.image.rgb_to_grayscale (image)
    image=tf.tile(image, [1,1,3])
  image = tf.image.random_brightness(image, max_delta=0.1)
  image = tf. image. random_contrast (image, lower=0.1, upper=0.2)
  image = tf.image.random_flip_left_right (image)
  return image, label

AUTOTUNE=tf.data.experimental.AUTOTUNE #tf allow us to give this to set automatically
BATCH_SIZE=32

ds_train=ds_train.map(normalize_image, num_parallel_calls=AUTOTUNE) # in normalize function these is no sequence so
#process can be done paralelly
ds_train= ds_train.cache() #it keep next data in memory so it will be fast
ds_train=ds_train.shuffle(ds_info.splits['train'].num_examples) #we can give 1000 so it will see 1000 datat points but
#it depends on records in a file so take from that


#ds_train=ds_train.map(augment, num_parallel_calls=AUTOTUNE)



ds_train=ds_train.batch(BATCH_SIZE)
ds_train=ds_train.prefetch(AUTOTUNE) #it keeps 64 examples ready when running on gpu the current one

ds_test=ds_test.map(normalize_image, num_parallel_calls=AUTOTUNE)
ds_test=ds_test.batch(128)
ds_test=ds_test.prefetch(AUTOTUNE)

class_names = [
"Airplane",
"Autmobile",
"Bird",
"Cat",
"Deep",
"Dog",
"Frog",
"Horse"
"Ship",
"Truck"
]

In [ ]:
d_train=ds_train.take(10)

In [ ]:
def train_model_one_epoch(hparams) :

  units = hparams[HP_NUM_UNITS]
  drop_rate = hparams[HP_DROPOUT]
  learning_rate = hparams[HP_LR]
  optimizer = keras.optimizers.Adam(lr=learning_rate)

  model = keras. Sequential (
      [
          layers. Input ((32, 32, 3)),
          layers. Conv2D(8, 3, padding ="same", activation="relu"),
          layers. Conv2D(16, 3, padding="same", activation="relu"),
          layers. MaxPooling2D((2, 2)),
          layers .Flatten (),
          layers. Dense (units, activation="relu"),
          layers.Dropout (drop_rate),
          layers. Dense (10),
          ]
      )

  for batch_idx, (x, y) in enumerate (d_train) :
    with tf.GradientTape () as tape:
      y_pred = model (x, training=True)
      loss = loss_fn(y, y_pred)
    gradients = tape.gradient (loss, model.trainable_weights)
    optimizer.apply_gradients(zip (gradients, model.trainable_weights))
    acc_metric.update_state(y, y_pred)

  #write to TB
  run_dir = (
  "logs/train/"
  +str (units)
  +"units " + str(drop_rate)
  +"dropout_" + str(learning_rate)
  +"learning_rate"
  )

  with tf.summary. create_file_writer(run_dir) .as_default ():
    hp.hparams (hparams)
    accuracy = acc_metric.result ()
    tf.summary. scalar ("accuracy", accuracy, step=1)
  acc_metric.reset_states ()

In [ ]:
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = keras.optimizers.Adam (lr=0.001)
acc_metric = keras.metrics. SparseCategoricalAccuracy ()
HP_NUM_UNITS = hp.HParam("num Units", hp.Discrete([32, 64, 128]))
HP_DROPOUT = hp.HParam("dropout", hp.Discrete([0.1, 0.2, 0.3, 0.5]))
HP_LR = hp. HParam("learning_rate", hp.Discrete([1e-3, 1e-4, 1e-5]))

In [ ]:
for lr in HP_LR.domain.values:
  for units in HP_NUM_UNITS.domain.values:
    for rate in HP_DROPOUT.domain.values:
      #we can put loop for number of epochs here
      hparams = {
          HP_LR : lr,
          HP_NUM_UNITS: units,
          HP_DROPOUT : rate,
      }
      train_model_one_epoch(hparams)

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir logs/

### Projector

In [ ]:
#in this we sent X, y batch and X feature vector so that using technique like tsne it prject that on dimension
#and show us the image where it is lying in the space

In [ ]:
#donot divide by 255 the pixel as not visible

In [ ]:
!rm -rf ./proj/

In [ ]:
(ds_train, ds_test), ds_info = tfds. load (
    "cifar10",
    split=["train", "test"],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,

)
def normalize_image(image, label):
  return tf.cast(image, tf.float32), label
def augment (image, label):
  new_height = new_width = 32

  image = tf.image.resize(image,(new_height, new_width))

  if tf.random.uniform((), minval=0, maxval=1) < 0.1:
    image = tf.image.rgb_to_grayscale (image)
    image=tf.tile(image, [1,1,3])
  image = tf.image.random_brightness(image, max_delta=0.1)
  image = tf. image. random_contrast (image, lower=0.1, upper=0.2)
  image = tf.image.random_flip_left_right (image)
  return image, label

AUTOTUNE=tf.data.experimental.AUTOTUNE #tf allow us to give this to set automatically
BATCH_SIZE=500

ds_train=ds_train.map(normalize_image, num_parallel_calls=AUTOTUNE) # in normalize function these is no sequence so
#process can be done paralelly
ds_train= ds_train.cache() #it keep next data in memory so it will be fast
ds_train=ds_train.shuffle(ds_info.splits['train'].num_examples) #we can give 1000 so it will see 1000 datat points but
#it depends on records in a file so take from that


#ds_train=ds_train.map(augment, num_parallel_calls=AUTOTUNE)



ds_train=ds_train.batch(BATCH_SIZE)
ds_train=ds_train.prefetch(AUTOTUNE) #it keeps 64 examples ready when running on gpu the current one

ds_test=ds_test.map(normalize_image, num_parallel_calls=AUTOTUNE)
ds_test=ds_test.batch(128)
ds_test=ds_test.prefetch(AUTOTUNE)

class_names = [
"Airplane",
"Autmobile",
"Bird",
"Cat",
"Deep",
"Dog",
"Frog",
"Horse"
"Ship",
"Truck",
"sdsd"
]

In [ ]:
import os
import cv2
import shutil
from tensorboard.plugins import projector

In [ ]:
def create_sprite (data) :
# For B&W or greyscale images
  if len (data.shape) == 3:
    data = np.tile(data[..., np.newaxis], (1, 1, 1, 3))

  n = int(np.ceil (np.sqrt (data.shape [0])))
  padding = ((0, n ** 2 - data. shape [0]), (0, 0), (0, 0), (0, 0))
  data = np.pad (data, padding, mode="constant", constant_values=0)
  # Tile images into sprite
  data = data.reshape ((n, n) + data.shape [1:]).transpose ((0, 2, 1, 3, 4))
  # print (data.shape) => (n, image_height, n, image_width, 3)

  data = data.reshape (
      (n * data. shape [1], n * data.shape [3]) + data. shape [4:]
      )
  # print (data .shape) => (n * image_height, n * image_width, 3)
  return data

In [ ]:
def plot_to_projector(
    x, feature_vector, y, class_names, log_dir="default_log_dir", meta_file="metadata.tsv",
    ) :
  assert x.ndim == 4 # (BATCH, H, W, C)
  if os.path.isdir(log_dir):
    shutil.rmtree (log_dir)
  #creating a folder
  os.mkdir (log_dir)

  SPRITES_FILE = os.path. join(log_dir, "sprites.png")
  sprite = create_sprite (x)
  cv2.imwrite (SPRITES_FILE, sprite)

  # Generate label names
  labels = [class_names[y[i]] for i in range(int(y.shape[0]))]
  with open(os.path. join(log_dir, meta_file), "w") as f:
    for label in labels:
      f.write (" {} \n" . format (label))

  if feature_vector.ndim != 2:
    print("NOTE: Feature vector is not of form (BATCH, FEATURES)"
    " reshaping to try and get it to this form!")
    feature_vector = tf. reshape (
    feature_vector, [feature_vector. shape[0], -1]
    )
    feature_vector = tf. Variable (feature_vector)
    checkpoint = tf. train.Checkpoint (embedding=feature_vector)
    checkpoint.save(os.path. join(log_dir, "embeddings.ckpt"))


  # Set up config
  config = projector. ProjectorConfig ()
  embedding = config.embeddings.add()
  embedding. tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
  embedding.metadata_path = meta_file


  embedding.sprite.image_path = "sprites.png"
  embedding.sprite.single_image_dim.extend ((x.shape [1], x. shape [2]))
  projector. visualize_embeddings(log_dir, config)

In [ ]:
x_batch, y_batch = next(iter(ds_train))
plot_to_projector(x_batch, x_batch, y_batch, class_names, log_dir="proj")

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir proj/

### profiler

In [ ]:
#in tensorflow 2 we have profiler that can do many things

In [ ]:
tf.__version__

In [ ]:
from datetime import datetime
from packaging import version
import os

In [ ]:
!pip install -U tensorboard-plugin-profile

In [ ]:
import tensorflow as tf

In [ ]:
import tensorflow_datasets as tfds
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train',
           'test'], shuffle_files=True, as_supervised=True, with_info=True,
)

In [ ]:
def normalize_img (image, label):
  return tf.cast(image, tf.float32)/255.0 , label

AUTOTUNE = tf.data.experimental.AUTOTUNE
ds_train = ds_train.map (normalize_img)
ds_train = ds_train.batch(128)
ds_test = ds_test. map (normalize_img)
ds_test = ds_test.batch(128)

model = tf. keras. models. Sequential ([
    tf.keras.layers. Flatten (input_shape= (28, 28, 1)),
    tf. keras. layers. Dense (128, activation='relu'),
    tf. keras.layers. Dense (10, activation='softmax')
])
model.compile (loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam (0.001), metrics= ['accuracy'])

In [ ]:
# Create a TensorBoard callback
logs = "logs/" + datetime.now ().strftime ("&Y&m§d-§H&M&S")

tensorboard_callback = tf.keras.callbacks.TensorBoard (log_dir = logs,
                                                       histogram_freq = 1,
                                                       profile_batch = '500, 520')
model.fit(ds_train,
          epochs=2,
          validation_data=ds_test,
          callbacks = [tensorboard_callback])

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir logs/